# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [4]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import folium

# Import API key
from api_keys import geoapify_key

In [5]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,saint-pierre,-21.3393,55.4781,24.76,68,100,3.09,RE,1697598114
1,1,port mathurin,-19.6833,63.4167,23.74,80,33,4.28,MU,1697598114
2,2,kudahuvadhoo,2.6708,72.8944,27.83,74,21,4.96,MV,1697598031
3,3,anadyr,64.7500,177.4833,-5.72,62,20,8.00,RU,1697598115
4,4,lazaro cardenas,17.9583,-102.2000,25.72,88,100,0.38,MX,1697598116


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [13]:
%%capture --no-display

# Configure the map plot
humidity_map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 500,
    frame_hight = 1000,
    size = "Humidity",
    color = "City"
)

# Display the map
humidity_map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [9]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_temp = (21,27)
ideal_windspeed = (0, 4.5)
max_cloudiness = 0

ideal_weather = city_data_df[
    (city_data_df["Max Temp"].between(ideal_temp[0], ideal_temp[1])) &
    (city_data_df["Wind Speed"].between(ideal_windspeed[0], ideal_windspeed[1])) &
    (city_data_df['Cloudiness'].fillna(0) <= max_cloudiness)
].dropna()

# Display sample data
ideal_weather

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
23,23,palmas,-10.2128,-48.3603,25.93,65,0,2.06,BR,1697597915
31,31,sheopur,25.6667,76.7000,24.92,60,0,1.32,IN,1697598126
51,51,remire-montjoly,4.9167,-52.2667,25.02,94,0,0.00,GF,1697598133
65,65,shimoda,34.6667,138.9500,23.20,61,0,2.69,JP,1697598138
86,86,santa maria da vitoria,-13.3947,-44.1886,23.18,44,0,1.10,BR,1697598144
133,133,shijiang,27.0819,110.7933,26.80,39,0,0.88,CN,1697598158
192,192,behbahan,30.5959,50.2417,22.21,25,0,1.56,IR,1697598175
271,271,dubbo,-32.2500,148.6167,24.07,31,0,3.60,AU,1697598199
273,273,layyah,30.9613,70.9390,21.71,54,0,1.70,PK,1697598199
300,300,meadow lake,34.8014,-106.5436,21.71,26,0,1.21,US,1697598207


### Step 3: Create a new DataFrame called `hotel_df`.

In [10]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
23,palmas,BR,-10.2128,-48.3603,65,
31,sheopur,IN,25.6667,76.7000,60,
51,remire-montjoly,GF,4.9167,-52.2667,94,
65,shimoda,JP,34.6667,138.9500,61,
86,santa maria da vitoria,BR,-13.3947,-44.1886,44,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [11]:
# Set parameters to search for a hotel
categories = "accommodation.hotel"
radius = 1000
params = {
    "categories":categories,
    "apiKey":geoapify_key 
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params).json()
    
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
palmas - nearest hotel: No hotel found
sheopur - nearest hotel: No hotel found
remire-montjoly - nearest hotel: No hotel found
shimoda - nearest hotel: ホテルウラガ
santa maria da vitoria - nearest hotel: No hotel found
shijiang - nearest hotel: No hotel found
behbahan - nearest hotel: No hotel found
dubbo - nearest hotel: Blue Gum Motel
layyah - nearest hotel: No hotel found
meadow lake - nearest hotel: No hotel found
arraial do cabo - nearest hotel: No hotel found
tazacorte - nearest hotel: No hotel found
zhuji - nearest hotel: No hotel found
uklana - nearest hotel: No hotel found
tura - nearest hotel: No hotel found
jeddah - nearest hotel: No hotel found
tindouf - nearest hotel: محمد بوسبي
nuriootpa - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
23,palmas,BR,-10.2128,-48.3603,65,No hotel found
31,sheopur,IN,25.6667,76.7000,60,No hotel found
51,remire-montjoly,GF,4.9167,-52.2667,94,No hotel found
65,shimoda,JP,34.6667,138.9500,61,ホテルウラガ
86,santa maria da vitoria,BR,-13.3947,-44.1886,44,No hotel found
133,shijiang,CN,27.0819,110.7933,39,No hotel found
192,behbahan,IR,30.5959,50.2417,25,No hotel found
271,dubbo,AU,-32.2500,148.6167,31,Blue Gum Motel
273,layyah,PK,30.9613,70.9390,54,No hotel found
300,meadow lake,US,34.8014,-106.5436,26,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [12]:
%%capture --no-display

# Configure the map plot
hotel_map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    color="City",
    size="Humidity",
    hover_cols=["Hotel Name", "Country"]
)

# Display the map
hotel_map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)